In [7]:
import pandas as pd
import os
import numpy as np

In [8]:
# Define the directory containing your CSV files
# place the link to the text file here
# directory = '/Users/martinso/Desktop/LST/HS-KI/Year 2/research trend/text'
directory = r'C:\Users\shwet\Documents\GitHub\SU_project\text'
data = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)

        # Initialize a list to store each row's text as a list of words
        texts = []

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Check for NaN and add text to the list after splitting into words
            if pd.notna(row['Text']):
                # Split the text into words and append the list of words
                texts.append(row['Text'].split())

        # Append the filename (without extension) and the list of texts to the data list
        data.append([os.path.splitext(filename)[0], texts])

# Create a DataFrame from the collected data
df_combined = pd.DataFrame(data, columns=['File Name', 'Text Data'])

# Define the path for the output Excel file
#output_file_path = os.path.join(directory, 'aggregated_texts_by-sentence.xlsx')

# Save the DataFrame to Excel
#df_combined.to_excel(output_file_path, index=False)


#print("All files have been processed and output to 'aggregated_texts.csv'.")

In [9]:
df_combined.rename(columns={'File Name': 'Name'}, inplace=True)
df_combined['Name'] = df_combined['Name'].str.replace('_Transcript', '')
df_combined['Name']=df_combined['Name'].astype(int)

df_combined.head()

,Name,Text Data
0,300,"[[so, I'm, going, to], [interview, in, Spanish..."
1,301,"[[yeah, there's, also, on, Craigslist, so, tha..."
2,302,"[[just, move, around, a, little, bit], [when, ..."
3,303,"[[wow, okay], [when, you're, finished, when, s..."
4,304,"[[so, we'll, just, move, around, a, little, bi..."


In [10]:
df = pd.read_csv('label_daic_extend - label_daic_extend.csv')
df.head()
df_filtered = df[['Participant_ID','depression']]

depression_map = df_filtered.set_index('Participant_ID')['depression']

df_combined['depression'] = df_combined['Name'].map(depression_map)

df_combined

,Name,Text Data,depression
0,300,"[[so, I'm, going, to], [interview, in, Spanish...",0
1,301,"[[yeah, there's, also, on, Craigslist, so, tha...",0
2,302,"[[just, move, around, a, little, bit], [when, ...",0
3,303,"[[wow, okay], [when, you're, finished, when, s...",0
4,304,"[[so, we'll, just, move, around, a, little, bi...",0
...,...,...,...
214,698,"[[going, to, press, that, button, it, just, me...",1
215,702,"[[hi], [I'm, not, a, therapist], [are, you, ok...",0
216,703,"[[and, please], [are, you, okay, with, this, y...",0
217,707,"[[okay, but, but, okay, but, I, don't, stop, u...",0


In [ ]:
# Function to calculate speech speed
def calculate_speech_speed(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Calculate the total duration in seconds for the participant (End_Time - Start_Time)
    total_duration = (df['End_Time'] - df['Start_Time']).sum()
    
    # Calculate the total number of words spoken (assuming the text column is named 'Text')
    total_words = df['Text'].str.split().apply(len).sum()
    
    # Avoid division by zero (in case of no spoken words or zero duration)
    if total_duration == 0:
        return np.nan
    else:
        return total_words / total_duration

# List to store participant speech speeds
speech_speeds = []

# Iterate over each file in the directory
for file_name in os.listdir(directory):
    if file_name.endswith('.csv'):
        # Get the full path of the CSV file
        file_path = os.path.join(directory, file_name)
        
        # Assuming the participant ID is in the file name (if not, adjust this as needed)
        participant_id = file_name.split('.')[0]  # Extracting participant ID from file name
        
        # Calculate the speech speed for this participant (unit: words/sec)
        speech_speed = calculate_speech_speed(file_path)
        
        # Append the result to the speech_speeds list
        speech_speeds.append({
            'Participant_ID': participant_id,
            'Speech_Speed': speech_speed
        })

# Convert the list to a DataFrame
df_temporary = pd.DataFrame(speech_speeds)
df_combined = pd.concat([df_combined, df_temporary], axis = 1).drop(columns = ["Participant_ID"])

df_combined.head()

,Name,Text Data,depression,Speech_Speed
0,300,"[[so, I'm, going, to], [interview, in, Spanish...",0,0.423796
1,301,"[[yeah, there's, also, on, Craigslist, so, tha...",0,1.058485
2,302,"[[just, move, around, a, little, bit], [when, ...",0,0.625064
3,303,"[[wow, okay], [when, you're, finished, when, s...",0,1.196229
4,304,"[[so, we'll, just, move, around, a, little, bi...",0,0.868119
